In [ ]:
# %cd models/bert
# !gdown 1nxVBajc4_thMTXOWdxstQnyzJ6SiqV8J  # 기본 KoBERT

# %cd models/poly_encoder
# !gdown 1VtB3fYNVb7I1dIIB4PuYVJpvQatePsLz  # 우리가 학습시킨 모델

# %cd models/cross_encoder
# !gdown 1-5Uei2q3uqbSPyMXzoSIj50l7dxIL2Kh  # 우리가 학습시킨 모델

In [ ]:
%cd  # Poly-Enocder folder로 가도록
from utils.model_for_inference import Load_Model_Tokenizer
import os
import pickle
import torch

CATEGORY = '' # normal, water, corona

poly_dir = 'path/to/poly_encoder'
cross_dir = 'path/to/cross_encoder'
emb_dir = 'datasets'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# q1, q2, q3, q4, text, embedding(text에 대한) 저장 pickle / 하지만 현재는 inference에 text와 embedding만 사용
with open(os.path.join(emb_dir, f'{CATEGORY}_with_text.pickle'), 'rb') as f:     
    embedding_df = pickle.load(f)

cross_encoder, _ = Load_Model_Tokenizer(cross_dir, model_type='cross')
poly_encoder, tokenizer = Load_Model_Tokenizer(poly_dir, model_type='poly')
cross_encoder.to(device)
poly_encoder.to(device)

In [ ]:
from utils.inference import Callcenter
import numpy as np 

call_center = Callcenter(poly_encoder=poly_encoder, cross_encoder=cross_encoder,
                        tokenizer=tokenizer, emb_df=embedding_df, device=device, topk=5)

query = '집에 가고 싶다.'
top_k_cross_scores, top_k_indices= call_center.inference(query)
# poly score가 가장 높은 k개의 답변 값들의 cross score 값과 index들 반환

top_cross_idx = top_k_indices[np.argmax(top_k_cross_scores)]
# k개의 답변들 중 cross score가 가장 높은 idx이용하여 최종 답변 산출
answer = embedding_df['text'].iloc[top_cross_idx]

print(f'질문 : {query}')
print(f'답변 : {answer}')